# 3.3: Split-apply-combine in pandas

Now that we are (somewhat) comfortable with transformation of data between wide and long, we can get into another very powerful pandas feature known as split-apply-combine.

---

## Dataset

We will again be using the dataset measuring "Machiavellianism" in people.

The three files are in your ```../assets/datasets/``` directory. They are:

- ```mach_data.csv``` which contains the wide data.
- ```mach_long.csv``` which contains the already long data.
- ```mach_codebook.csv``` which contains the information about the survey data.

---

## Packages

Loaded same as ever.

In [1]:
# data modules
import numpy as np
import scipy.stats as stats
import pandas as pd

# plotting modules
import matplotlib.pyplot as plt
import seaborn as sns

# make sure charts appear in the notebook:
%matplotlib inline

---

## A: Load the already widened data

I have already widened the data for you here in the interest of time. You can see how I get started on the bonus 2 question below as well.

If you would like to do the melting part yourself for practice, be my guest! The more practice the better. If doing it yourself you would instead load the ```mach_data.csv``` file again.

In [2]:
mach_long = pd.read_csv('../assets/datasets/mach_long.csv')
mach_long

,age,gender,subject_id,variable,value
0,24,1,1,Q1,4
1,33,2,2,Q1,2
2,21,1,3,Q1,3
3,17,1,4,Q1,4
4,22,1,5,Q1,4
5,21,1,6,Q1,5
6,55,1,7,Q1,4
7,40,1,8,Q1,1
8,30,2,9,Q1,3
9,65,1,10,Q1,2


In [3]:
print mach_long.variable.unique()

['Q1' 'Q2' 'Q3' 'Q4' 'Q5' 'Q6' 'Q7' 'Q8' 'Q9' 'Q10' 'Q11' 'Q12' 'Q13' 'Q14'
 'Q15' 'Q16' 'Q17' 'Q18' 'Q19' 'Q20' 'score' 'seconds_elapsed']


In [4]:
mach_long = mach_long[mach_long.variable != 'score']

In [5]:
seconds_data = mach_long.ix[mach_long.variable == 'seconds_elapsed', ['subject_id','variable','value']]
seconds_data['seconds_per_q'] = seconds_data.value / 20.
seconds_data

,subject_id,variable,value,seconds_per_q
255906,1,seconds_elapsed,177,8.85
255907,2,seconds_elapsed,107,5.35
255908,3,seconds_elapsed,323,16.15
255909,4,seconds_elapsed,136,6.80
255910,5,seconds_elapsed,142,7.10
255911,6,seconds_elapsed,144,7.20
255912,7,seconds_elapsed,123,6.15
255913,8,seconds_elapsed,196,9.80
255914,9,seconds_elapsed,128,6.40
255915,10,seconds_elapsed,168,8.40


### A.2 : Sorting by multiple columns with custom sort for questions

We are going to use ```pd.Categorical``` to custom-sort the questions after subject

see:  http://stackoverflow.com/questions/13838405/custom-sorting-in-pandas-dataframe

Look stuff up online!!

In [6]:
mach_long["variable"] = pd.Categorical(mach_long.variable, 
                                       ['Q1','Q2','Q3','Q4',
                                        'Q5','Q6','Q7','Q8',
                                        'Q9','Q10','Q11','Q12',
                                        'Q13','Q14','Q15','Q16',
                                        'Q17','Q18','Q19','Q20'])

/Users/pizzato/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
mach_long.sort_values(by=['subject_id','variable'])

,age,gender,subject_id,variable,value
0,24,1,1,Q1,4
12186,24,1,1,Q2,4
24372,24,1,1,Q3,2
36558,24,1,1,Q4,2
48744,24,1,1,Q5,4
60930,24,1,1,Q6,2
73116,24,1,1,Q7,3
85302,24,1,1,Q8,5
97488,24,1,1,Q9,3
109674,24,1,1,Q10,4


### A.3: Adding the time by question by subject_id as a new value

Merge the seconds data and the long data together. This is a preview to future lessons!

In [8]:
mach_secs = mach_long.merge(seconds_data[['subject_id', 'seconds_per_q']], on='subject_id', sort=False)
mach_secs

,age,gender,subject_id,variable,value,seconds_per_q
0,24,1,1,Q1,4,8.85
1,24,1,1,Q2,4,8.85
2,24,1,1,Q3,2,8.85
3,24,1,1,Q4,2,8.85
4,24,1,1,Q5,4,8.85
5,24,1,1,Q6,2,8.85
6,24,1,1,Q7,3,8.85
7,24,1,1,Q8,5,8.85
8,24,1,1,Q9,3,8.85
9,24,1,1,Q10,4,8.85


### A.4 Doing a cumulative sum of the seconds

Do a cumulative sum of the seconds by subject.

This is an example of the split-apply-combine pattern. We will do more as well.

#### A.4.1: Split the data into groups by subject_id

#### A.4.2: Apply the cumulative sum function, iterating through the groups

the ```.transform()``` function on group objects requires a function as the argument.

In [9]:
mach_subject_groups = mach_secs.groupby(by=['subject_id'])

In [10]:
mach_secs['seconds_elapsed'] = mach_subject_groups.seconds_per_q.transform(np.cumsum)
mach_secs

,age,gender,subject_id,variable,value,seconds_per_q,seconds_elapsed
0,24,1,1,Q1,4,8.85,8.85
1,24,1,1,Q2,4,8.85,17.70
2,24,1,1,Q3,2,8.85,26.55
3,24,1,1,Q4,2,8.85,35.40
4,24,1,1,Q5,4,8.85,44.25
5,24,1,1,Q6,2,8.85,53.10
6,24,1,1,Q7,3,8.85,61.95
7,24,1,1,Q8,5,8.85,70.80
8,24,1,1,Q9,3,8.85,79.65
9,24,1,1,Q10,4,8.85,88.50


In [11]:
mach_subject_groups.seconds_per_q.sum()

subject_id
1        185.85
2        112.35
3        339.15
4        142.80
5        149.10
6        151.20
7        129.15
8        205.80
9        134.40
10       176.40
11       156.45
12       151.20
13       264.60
14       156.45
15       169.05
16       118.65
17       326.55
18       199.50
19       136.50
20       267.75
21       307.65
22       162.75
23       228.90
24       124.95
25       375.90
26       178.50
27       175.35
28       159.60
29       223.65
30       337.05
          ...  
12157    120.75
12158    232.05
12159    161.70
12160    172.20
12161    202.65
12162    150.15
12163    200.55
12164    154.35
12165    139.65
12166    142.80
12167    249.90
12168    140.70
12169    290.85
12170    159.60
12171    163.80
12172    402.15
12173    208.95
12174    113.40
12175    111.30
12176    315.00
12177    175.35
12178    449.40
12179    193.20
12180    276.15
12181    403.20
12182    307.65
12183    200.55
12184    169.05
12185    285.60
12186    189.00
Name: seconds

### A.3: Make the re-widened dataset with TWO value columns

This is where the hierarchical index pattern really shines.

In [15]:
mach_secs_wide = pd.pivot_table(mach_secs,
                                values=['value','seconds_elapsed'],
                                index=['subject_id','age','gender'],
                                columns=['variable'])

In [17]:
mach_secs_wide.head()

value                                               \
variable                 Q1  Q10  Q11  Q12  Q13  Q14  Q15  Q16  Q17  Q18   
subject_id age gender                                                      
1          24  1        4.0  4.0  2.0  5.0  2.0  1.0  4.0  4.0  2.0  0.0   
2          33  2        2.0  4.0  2.0  4.0  2.0  2.0  3.0  2.0  2.0  4.0   
3          21  1        3.0  3.0  2.0  4.0  4.0  3.0  3.0  4.0  2.0  4.0   
4          17  1        4.0  3.0  1.0  5.0  3.0  2.0  4.0  2.0  2.0  3.0   
5          22  1        4.0  3.0  1.0  2.0  4.0  4.0  5.0  4.0  1.0  4.0   

                        ...   seconds_elapsed                            \
variable                ...               Q19    Q2    Q20     Q3    Q4   
subject_id age gender   ...                                               
1          24  1        ...            168.15  17.7  177.0  26.55  35.4   
2          33  2        ...            101.65  10.7  107.0  16.05  21.4   
3          21  1        ...            306.85  32.3  323.0  48.45  64.6   
4          17  1        ...            129.20  13.6  136.0  20.40  27.2   
5          22  1        ...            134.90  14.2  142.0  21.30  28.4   

                                                           
variable                  Q5    Q6      Q7     Q8      Q9  
subject_id age gender                                      
1          24  1       44.25  53.1   61.95   70.8   79.65  
2          33  2       26.75  32.1   37.45   42.8   48.15  
3          21  1       80.75  96.9  113.05  129.2  145.35  
4          17  1       34.00  40.8   47.60   54.4   61.20  
5          22  1       35.50  42.6   49.70   56.8   63.90  

[5 rows x 40 columns]

In [19]:
mach_secs_wide.index.names

FrozenList([u'subject_id', u'age', u'gender'])

In [24]:
mach_secs_wide[mach_secs_wide.index.get_level_values('age') == 24]

value                                               \
variable                 Q1  Q10  Q11  Q12  Q13  Q14  Q15  Q16  Q17  Q18   
subject_id age gender                                                      
1          24  1        4.0  4.0  2.0  5.0  2.0  1.0  4.0  4.0  2.0  0.0   
14         24  1        3.0  4.0  2.0  4.0  3.0  3.0  4.0  2.0  2.0  3.0   
22         24  1        5.0  2.0  2.0  5.0  2.0  3.0  2.0  2.0  2.0  4.0   
34         24  1        4.0  2.0  1.0  3.0  3.0  2.0  4.0  2.0  2.0  1.0   
96         24  2        5.0  5.0  1.0  5.0  5.0  2.0  4.0  1.0  2.0  4.0   
102        24  2        5.0  4.0  3.0  4.0  4.0  1.0  4.0  2.0  2.0  4.0   
115        24  1        3.0  4.0  1.0  1.0  2.0  2.0  3.0  5.0  3.0  4.0   
123        24  1        5.0  1.0  1.0  5.0  5.0  1.0  3.0  1.0  1.0  5.0   
127        24  2        2.0  4.0  3.0  2.0  1.0  3.0  3.0  4.0  4.0  2.0   
151        24  2        5.0  1.0  2.0  2.0  4.0  2.0  4.0  2.0  2.0  5.0   
153        24  1        4.0  2.0  4.0  5.0  4.0  2.0  4.0  2.0  2.0  4.0   
154        24  1        4.0  2.0  1.0  4.0  4.0  3.0  2.0  2.0  2.0  4.0   
155        24  1        4.0  2.0  1.0  4.0  4.0  3.0  2.0  2.0  2.0  4.0   
157        24  1        4.0  2.0  2.0  5.0  2.0  2.0  4.0  4.0  2.0  4.0   
178        24  2        2.0  5.0  2.0  3.0  2.0  3.0  3.0  5.0  4.0  4.0   
186        24  1        4.0  4.0  2.0  5.0  2.0  1.0  3.0  1.0  1.0  3.0   
208        24  1        3.0  4.0  1.0  4.0  2.0  2.0  5.0  1.0  2.0  4.0   
284        24  1        2.0  4.0  1.0  2.0  2.0  1.0  4.0  4.0  2.0  2.0   
285        24  1        4.0  3.0  1.0  3.0  5.0  2.0  4.0  1.0  1.0  5.0   
288        24  1        3.0  4.0  3.0  2.0  2.0  2.0  3.0  4.0  3.0  3.0   
293        24  2        4.0  5.0  1.0  4.0  1.0  1.0  4.0  1.0  2.0  5.0   
333        24  1        2.0  5.0  4.0  1.0  2.0  4.0  3.0  4.0  2.0  3.0   
336        24  1        4.0  0.0  1.0  5.0  4.0  1.0  0.0  5.0  1.0  4.0   
366        24  2        5.0  4.0  2.0  5.0  1.0  2.0  4.0  4.0  2.0  2.0   
369        24  1        2.0  4.0  4.0  4.0  4.0  2.0  2.0  2.0  3.0  2.0   
380        24  1        4.0  4.0  2.0  4.0  5.0  3.0  4.0  2.0  2.0  4.0   
383        24  1        4.0  1.0  4.0  4.0  4.0  4.0  2.0  5.0  3.0  1.0   
391        24  1        4.0  4.0  4.0  5.0  4.0  4.0  3.0  5.0  3.0  4.0   
396        24  1        0.0  3.0  3.0  3.0  1.0  3.0  3.0  4.0  3.0  4.0   
403        24  1        5.0  4.0  2.0  5.0  5.0  1.0  2.0  2.0  2.0  4.0   
...                     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
11624      24  1        4.0  2.0  3.0  4.0  4.0  2.0  4.0  2.0  3.0  4.0   
11637      24  2        4.0  2.0  1.0  2.0  3.0  1.0  2.0  5.0  2.0  1.0   
11648      24  1        2.0  4.0  4.0  4.0  2.0  3.0  3.0  3.0  4.0  3.0   
11664      24  1        3.0  1.0  2.0  5.0  3.0  2.0  4.0  4.0  1.0  4.0   
11671      24  1        4.0  2.0  2.0  4.0  3.0  2.0  3.0  2.0  4.0  3.0   
11676      24  2        3.0  4.0  1.0  2.0  3.0  1.0  4.0  5.0  1.0  4.0   
11679      24  1        4.0  3.0  1.0  4.0  4.0  2.0  4.0  4.0  2.0  4.0   
11695      24  2        2.0  4.0  2.0  5.0  2.0  2.0  3.0  4.0  2.0  4.0   
11697      24  1        3.0  2.0  2.0  4.0  4.0  3.0  3.0  2.0  2.0  3.0   
11700      24  1        1.0  4.0  1.0  2.0  4.0  1.0  4.0  5.0  2.0  5.0   
11701      24  2        5.0  1.0  2.0  2.0  2.0  2.0  5.0  2.0  2.0  4.0   
11776      24  1        3.0  4.0  3.0  3.0  2.0  3.0  3.0  5.0  3.0  1.0   
11780      24  2        2.0  2.0  1.0  5.0  2.0  1.0  4.0  1.0  1.0  4.0   
11800      24  1        4.0  2.0  2.0  4.0  5.0  1.0  2.0  2.0  1.0  4.0   
11811      24  2        3.0  2.0  2.0  4.0  5.0  1.0  5.0  2.0  2.0  4.0   
11812      24  2        4.0  2.0  1.0  4.0  4.0  2.0  4.0  3.0  2.0  4.0   
11841      24  1        2.0  3.0  2.0  4.0  3.0  3.0  4.0  3.0  3.0  4.0   
11855      24  1        5.0  2.0  1.0  5.0  4.0  1.0  5.0  3.0  1.0  0.0   
11856      24  2        3.0  4.0  2.0  4.0  2.0  2.0  4.0  4.0  2.0  3

In [29]:
mach_secs_wide.loc[pd.IndexSlice[:, 24:26, :], :].head()

value                                               \
variable                 Q1  Q10  Q11  Q12  Q13  Q14  Q15  Q16  Q17  Q18   
subject_id age gender                                                      
1          24  1        4.0  4.0  2.0  5.0  2.0  1.0  4.0  4.0  2.0  0.0   
14         24  1        3.0  4.0  2.0  4.0  3.0  3.0  4.0  2.0  2.0  3.0   
22         24  1        5.0  2.0  2.0  5.0  2.0  3.0  2.0  2.0  2.0  4.0   
25         26  2        5.0  3.0  1.0  3.0  1.0  1.0  3.0  3.0  2.0  3.0   
34         24  1        4.0  2.0  1.0  3.0  3.0  2.0  4.0  2.0  2.0  1.0   

                        ...   seconds_elapsed                            \
variable                ...               Q19    Q2    Q20     Q3    Q4   
subject_id age gender   ...                                               
1          24  1        ...            168.15  17.7  177.0  26.55  35.4   
14         24  1        ...            141.55  14.9  149.0  22.35  29.8   
22         24  1        ...            147.25  15.5  155.0  23.25  31.0   
25         26  2        ...            340.10  35.8  358.0  53.70  71.6   
34         24  1        ...            224.20  23.6  236.0  35.40  47.2   

                                                            
variable                  Q5     Q6      Q7     Q8      Q9  
subject_id age gender                                       
1          24  1       44.25   53.1   61.95   70.8   79.65  
14         24  1       37.25   44.7   52.15   59.6   67.05  
22         24  1       38.75   46.5   54.25   62.0   69.75  
25         26  2       89.50  107.4  125.30  143.2  161.10  
34         24  1       59.00   70.8   82.60   94.4  106.20  

[5 rows x 40 columns]

In [35]:
mach_secs_wide.xs(24, level='age', drop_level=False)

value                                               \
variable                 Q1  Q10  Q11  Q12  Q13  Q14  Q15  Q16  Q17  Q18   
subject_id age gender                                                      
1          24  1        4.0  4.0  2.0  5.0  2.0  1.0  4.0  4.0  2.0  0.0   
14         24  1        3.0  4.0  2.0  4.0  3.0  3.0  4.0  2.0  2.0  3.0   
22         24  1        5.0  2.0  2.0  5.0  2.0  3.0  2.0  2.0  2.0  4.0   
34         24  1        4.0  2.0  1.0  3.0  3.0  2.0  4.0  2.0  2.0  1.0   
96         24  2        5.0  5.0  1.0  5.0  5.0  2.0  4.0  1.0  2.0  4.0   
102        24  2        5.0  4.0  3.0  4.0  4.0  1.0  4.0  2.0  2.0  4.0   
115        24  1        3.0  4.0  1.0  1.0  2.0  2.0  3.0  5.0  3.0  4.0   
123        24  1        5.0  1.0  1.0  5.0  5.0  1.0  3.0  1.0  1.0  5.0   
127        24  2        2.0  4.0  3.0  2.0  1.0  3.0  3.0  4.0  4.0  2.0   
151        24  2        5.0  1.0  2.0  2.0  4.0  2.0  4.0  2.0  2.0  5.0   
153        24  1        4.0  2.0  4.0  5.0  4.0  2.0  4.0  2.0  2.0  4.0   
154        24  1        4.0  2.0  1.0  4.0  4.0  3.0  2.0  2.0  2.0  4.0   
155        24  1        4.0  2.0  1.0  4.0  4.0  3.0  2.0  2.0  2.0  4.0   
157        24  1        4.0  2.0  2.0  5.0  2.0  2.0  4.0  4.0  2.0  4.0   
178        24  2        2.0  5.0  2.0  3.0  2.0  3.0  3.0  5.0  4.0  4.0   
186        24  1        4.0  4.0  2.0  5.0  2.0  1.0  3.0  1.0  1.0  3.0   
208        24  1        3.0  4.0  1.0  4.0  2.0  2.0  5.0  1.0  2.0  4.0   
284        24  1        2.0  4.0  1.0  2.0  2.0  1.0  4.0  4.0  2.0  2.0   
285        24  1        4.0  3.0  1.0  3.0  5.0  2.0  4.0  1.0  1.0  5.0   
288        24  1        3.0  4.0  3.0  2.0  2.0  2.0  3.0  4.0  3.0  3.0   
293        24  2        4.0  5.0  1.0  4.0  1.0  1.0  4.0  1.0  2.0  5.0   
333        24  1        2.0  5.0  4.0  1.0  2.0  4.0  3.0  4.0  2.0  3.0   
336        24  1        4.0  0.0  1.0  5.0  4.0  1.0  0.0  5.0  1.0  4.0   
366        24  2        5.0  4.0  2.0  5.0  1.0  2.0  4.0  4.0  2.0  2.0   
369        24  1        2.0  4.0  4.0  4.0  4.0  2.0  2.0  2.0  3.0  2.0   
380        24  1        4.0  4.0  2.0  4.0  5.0  3.0  4.0  2.0  2.0  4.0   
383        24  1        4.0  1.0  4.0  4.0  4.0  4.0  2.0  5.0  3.0  1.0   
391        24  1        4.0  4.0  4.0  5.0  4.0  4.0  3.0  5.0  3.0  4.0   
396        24  1        0.0  3.0  3.0  3.0  1.0  3.0  3.0  4.0  3.0  4.0   
403        24  1        5.0  4.0  2.0  5.0  5.0  1.0  2.0  2.0  2.0  4.0   
...                     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
11624      24  1        4.0  2.0  3.0  4.0  4.0  2.0  4.0  2.0  3.0  4.0   
11637      24  2        4.0  2.0  1.0  2.0  3.0  1.0  2.0  5.0  2.0  1.0   
11648      24  1        2.0  4.0  4.0  4.0  2.0  3.0  3.0  3.0  4.0  3.0   
11664      24  1        3.0  1.0  2.0  5.0  3.0  2.0  4.0  4.0  1.0  4.0   
11671      24  1        4.0  2.0  2.0  4.0  3.0  2.0  3.0  2.0  4.0  3.0   
11676      24  2        3.0  4.0  1.0  2.0  3.0  1.0  4.0  5.0  1.0  4.0   
11679      24  1        4.0  3.0  1.0  4.0  4.0  2.0  4.0  4.0  2.0  4.0   
11695      24  2        2.0  4.0  2.0  5.0  2.0  2.0  3.0  4.0  2.0  4.0   
11697      24  1        3.0  2.0  2.0  4.0  4.0  3.0  3.0  2.0  2.0  3.0   
11700      24  1        1.0  4.0  1.0  2.0  4.0  1.0  4.0  5.0  2.0  5.0   
11701      24  2        5.0  1.0  2.0  2.0  2.0  2.0  5.0  2.0  2.0  4.0   
11776      24  1        3.0  4.0  3.0  3.0  2.0  3.0  3.0  5.0  3.0  1.0   
11780      24  2        2.0  2.0  1.0  5.0  2.0  1.0  4.0  1.0  1.0  4.0   
11800      24  1        4.0  2.0  2.0  4.0  5.0  1.0  2.0  2.0  1.0  4.0   
11811      24  2        3.0  2.0  2.0  4.0  5.0  1.0  5.0  2.0  2.0  4.0   
11812      24  2        4.0  2.0  1.0  4.0  4.0  2.0  4.0  3.0  2.0  4.0   
11841      24  1        2.0  3.0  2.0  4.0  3.0  3.0  4.0  3.0  3.0  4.0   
11855      24  1        5.0  2.0  1.0  5.0  4.0  1.0  5.0  3.0  1.0  0.0   
11856      24  2        3.0  4.0  2.0  4.0  2.0  2.0  4.0  4.0  2.0  3